In [72]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import warnings
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from IPython.display import display
import os

In [73]:
df = pd.read_excel("D:\IBM ML\data\LAB - 2\Sample - Superstore.xls")
df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit'],
      dtype='object')

We can see the data have been input and the columns are referenced by a pandas index object. there are two date variable (order date and ship date),variables for customer and region , product type variable ((category , sub-category , product name)),etc.

SIMPLIFY TIME SERIES DATA 
We will start by simplify the input data a bit to explore data type . To do so , we will look at total sales by order Date and Category . This allows us to look a Time series dataset with multipletime series.

In [74]:
variables = ['Order Date', 'Category','Sales']
group_variables = variables[:2]
outcome_variable = variables[2]
base = df.groupby(group_variables)[outcome_variable].sum().reset_index()

In [75]:
print("Columns:", base.columns)
print("Index:",base.index)

Columns: Index(['Order Date', 'Category', 'Sales'], dtype='object')
Index: RangeIndex(start=0, stop=2864, step=1)


INDIVIDUAL DATAFRAME COLUMNS ARE PANDAS SERIES, AND WE CAN SEE THE RANGE INDEX ON THE LEFT . THIS PANDAS DATAFRAME IS A COMBINATION OF THE RANGE INDEX AND PANDAS SERIES OBJECTS, WHERE EACH HAS AN UNDERLYING DATATYPE:

In [76]:
base.head()

,Order Date,Category,Sales
0,2011-01-04,Office Supplies,16.448
1,2011-01-05,Office Supplies,288.060
2,2011-01-06,Office Supplies,19.536
3,2011-01-07,Furniture,2573.820
4,2011-01-07,Office Supplies,685.340


In [77]:
base.dtypes

Order Date    datetime64[ns]
Category              object
Sales                float64
dtype: object

PANDAS DATAFRAME TYPES:

In [78]:
for x in base.columns:
    print(x, type(base[x]), base[x].dtype)

Order Date <class 'pandas.core.series.Series'> datetime64[ns]
Category <class 'pandas.core.series.Series'> object
Sales <class 'pandas.core.series.Series'> float64


WORKING WITH NUMPY ARRAYS 

It isn't always necessary to extract Numpy array , as pandas series contain Numpy functionality . However some applications use Numpy array as inputs and can bypass pandas if desired

In [79]:
order_date = np.array([base['Order Date']])
category  = np.array(base['Category'])
sales = np.array(base['Sales'])

In [80]:
print('Order Date',type (order_date),order_date.dtype)
print('Category', type(category),category.dtype)
print('sales', type(sales),sales.dtype)

Order Date <class 'numpy.ndarray'> datetime64[ns]
Category <class 'numpy.ndarray'> object
sales <class 'numpy.ndarray'> float64


If starting from the NumPy arrays , we could bulid the dataframe (note dictionary) input structure:

In [81]:
order_date

array([['2011-01-04T00:00:00.000000000', '2011-01-05T00:00:00.000000000',
        '2011-01-06T00:00:00.000000000', ...,
        '2014-12-31T00:00:00.000000000', '2014-12-31T00:00:00.000000000',
        '2014-12-31T00:00:00.000000000']], dtype='datetime64[ns]')

In [82]:
order_date_daily = np.array(order_date, dtype = 'datetime64[D]')
order_date_daily

array([['2011-01-04', '2011-01-05', '2011-01-06', ..., '2014-12-31',
        '2014-12-31', '2014-12-31']], dtype='datetime64[D]')

In [83]:
order_date_monthly = np.array(order_date, dtype = 'datetime64[M]')
order_date_monthly

array([['2011-01', '2011-01', '2011-01', ..., '2014-12', '2014-12',
        '2014-12']], dtype='datetime64[M]')

In [84]:
np.unique(order_date_monthly)

array(['2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
       '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
       '2012-01', '2012-02', '2012-03', '2012-04', '2012-05', '2012-06',
       '2012-07', '2012-08', '2012-09', '2012-10', '2012-11', '2012-12',
       '2013-01', '2013-02', '2013-03', '2013-04', '2013-05', '2013-06',
       '2013-07', '2013-08', '2013-09', '2013-10', '2013-11', '2013-12',
       '2014-01', '2014-02', '2014-03', '2014-04', '2014-05', '2014-06',
       '2014-07', '2014-08', '2014-09', '2014-10', '2014-11', '2014-12'],
      dtype='datetime64[M]')

WORKING WITH PANDAS DATETIMEINDEX


In [85]:
print(base.head())
print('\n Unique categories:')
print(base['Category'].unique())

  Order Date         Category     Sales
0 2011-01-04  Office Supplies    16.448
1 2011-01-05  Office Supplies   288.060
2 2011-01-06  Office Supplies    19.536
3 2011-01-07        Furniture  2573.820
4 2011-01-07  Office Supplies   685.340

 Unique categories:
['Office Supplies' 'Furniture' 'Technology']


SETTING INDEX USING EXISTING VARIABLE 

we often want to set an index explicity , or manipulate an index , for working with time series data . The pandas date time index is useful here , although it is often useful to standardize the index by ensuring all relevant time period are include only once . Our data violate this condition for two reasons: (1) Multiple values for a given period (due to multiple categories) and (2) missing days (for daily). We will fix both of these issues below , and explore some useful datetime functionality.

In [86]:
base.set_index('Order Date', inplace=True)
# Note that without inplace = True , it will output the results without changing the data 

In [87]:
base.head()

,Category,Sales
Order Date,,
2011-01-04,Office Supplies,16.448
2011-01-05,Office Supplies,288.060
2011-01-06,Office Supplies,19.536
2011-01-07,Furniture,2573.820
2011-01-07,Office Supplies,685.340


In [88]:
print(base.index)
# print (base.index.unique())

DatetimeIndex(['2011-01-04', '2011-01-05', '2011-01-06', '2011-01-07',
               '2011-01-07', '2011-01-07', '2011-01-08', '2011-01-08',
               '2011-01-10', '2011-01-10',
               ...
               '2014-12-28', '2014-12-29', '2014-12-29', '2014-12-29',
               '2014-12-30', '2014-12-30', '2014-12-30', '2014-12-31',
               '2014-12-31', '2014-12-31'],
              dtype='datetime64[ns]', name='Order Date', length=2864, freq=None)


SUBSETTING DATA

We now has Datetimeindex and we can use it to select data subset 

In [89]:
# Observations in 2014
display(base['2011'].head())
print('\n')
#Observations in a range of dates subset of columns:
display(base[base['Category'] == 'Office Supplies']['2011':'2012-02'].tail())

C:\Users\arjun\AppData\Local\Temp\ipykernel_19792\3664258241.py:2: FutureWarning: Indexing a DataFrame with a datetimelike index using a single string to slice the rows, like `frame[string]`, is deprecated and will be removed in a future version. Use `frame.loc[string]` instead.
  display(base['2011'].head())


,Category,Sales
Order Date,,
2011-01-04,Office Supplies,16.448
2011-01-05,Office Supplies,288.060
2011-01-06,Office Supplies,19.536
2011-01-07,Furniture,2573.820
2011-01-07,Office Supplies,685.340


,Category,Sales
Order Date,,
2012-02-22,Office Supplies,79.360
2012-02-23,Office Supplies,37.776
2012-02-25,Office Supplies,25.870
2012-02-27,Office Supplies,20.451
2012-02-29,Office Supplies,58.720


DATETIME COMPONENTS

Pandas datetime variable have a number of useful components. Using the Datetimeindex , we can extract items like month, year , day of week , quater, etc:

In [90]:
# base.set_index('OrderDate', inplace == True)
print('Day:', base.index.day, '\n')
print('Week:', base.index.week, '\n')
base['DayofWeek'] = base.index.dayofweek # day of week : monday = 0, sunday = 6
print(base.head())
# Note: use dt method when the data variable is not part of the index:
# df('Order Date').dt.dayofweek.head()
del(base['DayofWeek'])

Day: Int64Index([ 4,  5,  6,  7,  7,  7,  8,  8, 10, 10,
            ...
            28, 29, 29, 29, 30, 30, 30, 31, 31, 31],
           dtype='int64', name='Order Date', length=2864) 

Week: Int64Index([ 1,  1,  1,  1,  1,  1,  1,  1,  2,  2,
            ...
            52,  1,  1,  1,  1,  1,  1,  1,  1,  1],
           dtype='int64', name='Order Date', length=2864) 

                   Category     Sales  DayofWeek
Order Date                                      
2011-01-04  Office Supplies    16.448          1
2011-01-05  Office Supplies   288.060          2
2011-01-06  Office Supplies    19.536          3
2011-01-07        Furniture  2573.820          4
2011-01-07  Office Supplies   685.340          4


C:\Users\arjun\AppData\Local\Temp\ipykernel_19792\1250898062.py:3: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series. To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  print('Week:', base.index.week, '\n')


STANDARDIZING THE DATETIMEINDEX 

While data from existing variable may be sufficient , some time series applications requires that data contain all periods and have a frequency assigned. We can see above that our data do not have a frequency (freq = none). while the data seem daily, there are many types of possible frequencies. Pandas will infer a frequency and assign it. Otherwise , we need to ensure there are:

1. No duplicate index values
2. No missing index values 

PIVOTING DATA:

Because there are multiple categories , we have multiple time series to analyze . As a result , our datetimeindex does not uniquely identify an observation . To uniquely identify observations, we can either add categorical variable to the index , or set a pandas date timeindex with separate columns for each series . There are several ways to accomplish this . The first approach uses pandas bulit-in pivot method:

In [91]:
base.reset_index(inplace=True)
# Note if we didn't reset the index , we could use index = none below
sales_pivot = base.pivot(index='Order Date', columns = 'Category', values= 'Sales')
sales_pivot.head

<bound method NDFrame.head of Category    Furniture  Office Supplies  Technology
Order Date                                        
2011-01-04        NaN           16.448         NaN
2011-01-05        NaN          288.060         NaN
2011-01-06        NaN           19.536         NaN
2011-01-07  2573.8200          685.340    1147.940
2011-01-08    76.7280           10.430         NaN
...               ...              ...         ...
2014-12-27        NaN          814.594         NaN
2014-12-28        NaN           13.248     164.388
2014-12-29   551.2568         1091.244      14.850
2014-12-30  2330.7180          282.440     302.376
2014-12-31   323.1360          299.724      90.930

[1238 rows x 3 columns]>

UNSTACKING: 

To achieve the same result in pandas, it is often easier to use the index and unstack / stack methods. the unstack method tranforms long data into wide data by creating columns by category for levels of the index , while stack does the reverse.

In [92]:
sales = base.set_index(['Order Date', 'Category']).unstack('Category').fillna(0)
sales.columns = sales.columns.levels[1].rename(None)
sales.head()


,Furniture,Office Supplies,Technology
Order Date,,,
2011-01-04,0.000,16.448,0.00
2011-01-05,0.000,288.060,0.00
2011-01-06,0.000,19.536,0.00
2011-01-07,2573.820,685.340,1147.94
2011-01-08,76.728,10.430,0.00


Generating a complete index and setting frequency 

since we are using daily data , we would like to set a daily frequency. We see our data has a length of 1238 days. By subtracting the smallest date from the largest date , we can tell there are some days missing:

In [96]:
print('\nUnique dates in our data: ', len(sales.index.unique()),'Days')
our_data_range = sales.index.max() - sales.index.min()
print('Total days in our date range:', our_data_range.days,'Days')



Unique dates in our data:  1238 Days
Total days in our date range: 1457 Days


In [97]:
new_index = pd.date_range(sales.index.min(), sales.index.max())
print(new_index)

DatetimeIndex(['2011-01-04', '2011-01-05', '2011-01-06', '2011-01-07',
               '2011-01-08', '2011-01-09', '2011-01-10', '2011-01-11',
               '2011-01-12', '2011-01-13',
               ...
               '2014-12-22', '2014-12-23', '2014-12-24', '2014-12-25',
               '2014-12-26', '2014-12-27', '2014-12-28', '2014-12-29',
               '2014-12-30', '2014-12-31'],
              dtype='datetime64[ns]', length=1458, freq='D')
